# Imports

In [1]:
import boto3
import json
import numpy as np
from keras.models import load_model

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Utility Classes

In [2]:
class AWSUtilsLib():
    
    def __init__(self, aws_access_key, aws_secret_key, region_id = ''):
        self.sqs                    = None
        self.queue                  = None
        self.queue_url              = '' 
        self.acess_key              = aws_access_key
        self.secret_key             = aws_secret_key
        self.region                 = region_id
        
    #######################
    #### SQS FUNCTIONS ####
    #######################
    def create_sqs_client(self, region_id = ''):
        
        if region_id:
            self.region = region_id
            
        self.sqs = boto3.resource('sqs',
             aws_access_key_id     = self.acess_key,
             aws_secret_access_key = self.secret_key,
             region_name           = self.region)

    def create_queue_client(self, queuename):

        # Get Queue URL
        self.queue_url = self.sqs.get_queue_by_name(QueueName=queuename)

        # Create Queue Resource
        self.queue = sqs.Queue(self.queue_url.url)
        

        
        


# Code

In [21]:
# Get sqs and queue client
aws_in = AWSUtilsLib('access','secret', 'us-east-1')
aws_out = AWSUtilsLib('access','secret', 'us-east-1')
aws_in.create_sqs_client()
aws_in.create_queue_client('LUNAR_ToBeTested')
aws_out.create_sqs_client()
aws_out.create_queue_client('LUNAR_Processed')


# Load machine learning model
model = load_model('LUNAR_MachineLearningModel.h5')


msgs       = []
samples    = []
counter    = 0
batch_size = 60

while True:
    
    # Get messages
    messages = aws_in.queue.receive_messages(MaxNumberOfMessages=10, WaitTimeSeconds=5, AttributeNames=['All'])
    
    # Iterate over messages
    for message in messages:

        # Trace msg
        if counter % batch_size == 0:
            print ('Already processed {} messages'.format(counter))
        
        # Deserialize object
        data = json.load(message.body)
        
        samples.append([data['Accelerometer_X'],
                        data['Accelerometer_Y'],
                        data['Accelerometer_Z']])
        
        msgs.append(message)
        
        if (len(samples) == batch_size):
            
            # Restructure the object to be readable by the saved model
            restructured_obj = [samples]
            
            # Applies the machine learning model
            y_pred = model.predict(restructured_obj)
            output = np.argmax(y_pred, axis=1)[0]
            
            # Save the predicted output on record
            data['Output'] = str(output)
        
            # Send to next queue (LUNAR_Processed)
            response = aws_out.queue.send_message(QueueUrl=aws.queue_url, MessageBody=data)
        
            # Delete msgs
            for msg in msgs:
                msg.delete()
            
            msgs    = []
            samples = []
        
        counter += 1

SyntaxError: 'return' outside function (<ipython-input-21-abb634ce1f71>, line 2)